**Paso 0: Librerias**

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

import joblib

**Paso 1: Limpieza avanzada y estandarización de variables**

In [2]:
# 📌 LIMPIEZA Y RENOMBRADO DE VARIABLES CLAVE
df = pd.read_csv(r"C:\Users\javid\OneDrive\Escritorio\Javidev\ProjectML_Prediccion_Salarios_IT\data\survey_results_public.csv", low_memory=False)

# Renombramos las columnas más importantes
df = df.rename(columns={
    "Employment": "Tipo_empleo",
    "RemoteWork": "Trabajo_remoto",
    "DevType": "Rol",
    "EdLevel": "Nivel_educativo",
    "YearsCodePro": "Anios_experiencia",
    "Country": "Pais",
    "OrgSize": "Tamano_empresa",
    "ConvertedCompYearly": "Salario_anual"
})

# Filtramos columnas relevantes
columnas_utiles = [
    "Tipo_empleo", "Trabajo_remoto", "Rol", "Nivel_educativo",
    "Anios_experiencia", "Pais", "Tamano_empresa", "Salario_anual"
]
df = df[columnas_utiles]

# Limpiamos y transformamos la experiencia profesional
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0",
    "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")

# Agrupamos países poco frecuentes como "Otro"
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Convertimos tipo_empleo en categorías resumidas
def simplificar_tipo_empleo(valor):
    if pd.isna(valor): return None
    if "full-time" in valor: return "Jornada completa"
    if "part-time" in valor: return "Media jornada"
    if "freelancer" in valor or "contractor" in valor: return "Autonomo"
    if "student" in valor: return "Estudiante"
    if "not employed" in valor: return "Desempleado"
    if "retired" in valor: return "Jubilado"
    return "Otro"

df["Tipo_empleo"] = df["Tipo_empleo"].apply(simplificar_tipo_empleo)

# Simplificamos modalidad de trabajo
df["Trabajo_remoto"] = df["Trabajo_remoto"].replace({
    "Remote": "Remoto",
    "In-person": "Presencial",
    "Hybrid (some remote, some in-person)": "Hibrido"
})

def mapear_roles(rol_raw):
    mapa_roles = {
        "Developer, full-stack": "Desarrollador Full Stack",
        "Developer, back-end": "Desarrollador Back-End",
        "Developer, front-end": "Desarrollador Front-End",
        "Developer, desktop or enterprise applications": "Desarrollador de Aplicaciones de Escritorio o Empresariales",
        "Developer, mobile": "Desarrollador Mobile",
        "Developer, embedded applications or devices": "Desarrollador de Sistemas Embebidos",
        "Other (please specify)": "Otro",
        "Data engineer": "Ingeniero de Datos",
        "Engineering manager": "Manager de Ingenieria",
        "DevOps specialist": "Especialista DevOps",
        "Data scientist or machine learning specialist": "Cientifico de Datos / Especialista en ML",
        "Research & Development role": "Investigación y Desarrollo",
        "Academic researcher": "Investigador Academico",
        "Cloud infrastructure engineer": "Ingeniero de Infraestructura Cloud",
        "Senior Executive (C-Suite, VP, etc.)": "Directivo / Alta Direccion"
    }

    if pd.isna(rol_raw):
        return []
    
    roles = [r.strip() for r in rol_raw.split(";")]
    roles_mapeados = [mapa_roles.get(r, "Otro") for r in roles]
    return roles_mapeados

df["Rol"] = df["Rol"].apply(mapear_roles)

# Simplificamos nivel educativo
def simplificar_nivel(nivel):
    if pd.isna(nivel): return None
    if "Bachelor" in nivel: return "Grado universitario"
    if "Master" in nivel: return "Master"
    if "Professional" in nivel or "Ph.D" in nivel or "Doctoral" in nivel: return "Doctorado"
    if "Secondary" in nivel: return "Secundaria"
    if "Primary" in nivel: return "Primaria"
    if "Associate" in nivel: return "Grado medio"
    if "Some college" in nivel: return "Universidad sin titulo"
    return "Otro"

df["Nivel_educativo"] = df["Nivel_educativo"].apply(simplificar_nivel)

# Eliminamos filas nulas y salarios excesivos
# df = df.dropna()
df = df[df["Salario_anual"] <= 300000]


**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [3]:
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")
df["Tamano_empresa"] = df["Tamano_empresa"].replace("I don’t know", pd.NA)

In [4]:
df = df.dropna()
df = df[df["Salario_anual"] <= 300000]

**Paso 2: Procesamiento de la columna 'Rol' y 'Pais' y 'Tamano_empresa'**

In [5]:
# Rol
roles_dummies = df["Rol"].str.get_dummies(sep=";")
top_roles = roles_dummies.sum().sort_values(ascending=False).head(15).index
df = pd.concat([df.drop(columns="Rol"), roles_dummies[top_roles]], axis=1)

# País
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")
df = pd.get_dummies(df, columns=["Pais"], drop_first=False)
if "Pais_Otro" in df.columns:
    df = df.drop(columns=["Pais_Otro"])

# Resto
df = pd.get_dummies(df, columns=["Tipo_empleo", "Trabajo_remoto", "Nivel_educativo"], drop_first=False)

orden_empresa = [
    "Just me - I am a freelancer, sole proprietor, etc.",
    "2 to 9 employees",
    "10 to 19 employees",
    "20 to 99 employees",
    "100 to 499 employees",
    "500 to 999 employees",
    "1,000 to 4,999 employees",
    "5,000 to 9,999 employees",
    "10,000 or more employees"
]
df["Tamano_empresa"] = df["Tamano_empresa"].apply(lambda x: orden_empresa.index(x))

**Paso 3: Codificación de columnas categóricas**

In [6]:
for col in df.columns:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)

**Paso 4: Limpieza y preparación final**

In [7]:
# Preparar datos finales
X = df.drop(columns="Salario_anual")
X.columns = [re.sub(r"[^a-zA-Z0-9]", "_", col) for col in X.columns]
y = df["Salario_anual"]

**Paso 5: División en train/test**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

**Paso 6: Definición del grid de hiperparámetros**

In [9]:
param_grid = {
    "iterations": [800],
    "learning_rate": [0.03],
    "depth": [6],
    "l2_leaf_reg": [3],
    "bagging_temperature": [0.2]
}

**Paso 7: GridSearchCV y entrenamiento de CatBoost**

In [10]:
model = CatBoostRegressor(verbose=0, random_state=42)

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="r2",
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Mejor configuración encontrada:")
print(grid.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mejor configuración encontrada:
{'bagging_temperature': 0.2, 'depth': 6, 'iterations': 800, 'l2_leaf_reg': 3, 'learning_rate': 0.03}


**Paso 8: Evaluación contra test**

In [11]:
# Evaluación en test
mejor_modelo = grid.best_estimator_
y_pred = mejor_modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)

print(f"R² en test con mejores hiperparámetros: {r2:.4f}")


R² en test con mejores hiperparámetros: 0.5901


**Paso 9: Guardado de modelo**

In [12]:
joblib.dump(mejor_modelo, "model.pkl")

['model.pkl']

In [14]:
list(X.head().columns)

['Anios_experiencia',
 'Tamano_empresa',
 '__Desarrollador_Full_Stack__',
 '__Desarrollador_Back_End__',
 '__Otro__',
 '__Desarrollador_Front_End__',
 '__Desarrollador_de_Aplicaciones_de_Escritorio_o_Empresariales__',
 '__Desarrollador_Mobile__',
 '__Desarrollador_de_Sistemas_Embebidos__',
 '__Ingeniero_de_Datos__',
 '__Manager_de_Ingenieria__',
 '__Especialista_DevOps__',
 '__Cientifico_de_Datos___Especialista_en_ML__',
 '__Investigaci_n_y_Desarrollo__',
 '__Investigador_Academico__',
 '__Ingeniero_de_Infraestructura_Cloud__',
 '__Directivo___Alta_Direccion__',
 'Pais_Australia',
 'Pais_Brazil',
 'Pais_Canada',
 'Pais_France',
 'Pais_Germany',
 'Pais_India',
 'Pais_Italy',
 'Pais_Netherlands',
 'Pais_Poland',
 'Pais_Spain',
 'Pais_Sweden',
 'Pais_Ukraine',
 'Pais_United_Kingdom_of_Great_Britain_and_Northern_Ireland',
 'Pais_United_States_of_America',
 'Tipo_empleo_Autonomo',
 'Tipo_empleo_Jornada_completa',
 'Tipo_empleo_Media_jornada',
 'Trabajo_remoto_Hibrido',
 'Trabajo_remoto_Pres